In [1]:
import random
import tensorflow as tf
import numpy as np

In [53]:
symbol = {
    '+':1000,
    '-':2000,
    '*':3000,
    '/':4000,
}

symbol_s = {
    1000:'+',
    2000:'-',
    3000:'*',
    4000:'/',
}

In [5]:
upper_bound=10
lower_bound=1

def generateEquation():
    eq = []
    eq.append(str(random.randint(lower_bound, upper_bound)))
    for i in range(4):
        eq.append(symbol_s[random.randint(1, 4)*1000])
        eq.append(str(random.randint(lower_bound, upper_bound)))
    return eq

In [6]:
test = generateEquation()
print(''.join(test))

5/8*5-3+9


In [7]:
def generateStep(a):  
    mark=[] # step1 marks
    sol = [a]
    
    n = len(a)//2
    
    prev = a
    for i in range(n-1,-1,-1):
        x = random.randint(0,i)*2
        #print("i="+str(i)+"x="+str(x))
        #print(prev[:2*x],prev[2*x+3:])
        
        middle = x+1
        while prev[middle]=='':
            middle+=1;
        
        start = middle-1
        while prev[start]=='':
            start-=1;
            
        end = middle+1
        while prev[end]=='':
            end+=1;
        #print(start,middle,end)
        
        #prefix
        step = prev[:start]
        #process step
        
        cal_value = eval(''.join(prev[start:end+1]))
        step.append(str(cal_value))
        #suffix
        step += ['']*(end-start)
        step += prev[end+1:]
        sol.append(step)
        
        #print(step)
        #print(''.join(prev[x:x+3]))
        
        #give mark
        sym = a[middle]
        if sym == '+':
            mark.append(0) if ('*' in prev) or ('/' in prev) else mark.append(1)
        elif sym == '-':
            mark.append(0) if ('*' in prev) or ('/' in prev) else mark.append(1)
        else:
            mark.append(1)

        
        
        prev = step
    
    
    
    
    
    return (sol,mark)

In [8]:
test = generateEquation()
(sol,mark) = generateStep(test)
for step in sol:
    print(len(step))

9
9
9
9
9


In [9]:
def Token(a):
    _a = []
    for x in range(2):
        _step = []
        for i in a[x]:
            if i == '':
                _i=0
            else:
                try:
                    _i = float(i)
                except:
                    _i = symbol[i]
            _step.append(_i)
        _a.append(_step)
    return _a

In [23]:
size = 1000

traindata=[]
trainlabel=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            traindata.append(Token(sol[j:j+2]))
        trainlabel+=mark
    except:
        i=i-1

print(len(traindata),len(trainlabel))
#print(traindata)
#print('\n')
#print(trainlabel)

3964 3964


In [24]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(36, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit(traindata, trainlabel, epochs=10)

Epoch 1/10
124/124 [==============================] - 0s 2ms/step - loss: 44.5659 - accuracy: 0.7861
Epoch 2/10
124/124 [==============================] - 0s 2ms/step - loss: 4.6731 - accuracy: 0.9092
Epoch 3/10
124/124 [==============================] - 0s 2ms/step - loss: 3.7993 - accuracy: 0.9190
Epoch 4/10
124/124 [==============================] - 0s 2ms/step - loss: 2.9785 - accuracy: 0.9263
Epoch 5/10
124/124 [==============================] - 0s 2ms/step - loss: 3.4497 - accuracy: 0.9228
Epoch 6/10
124/124 [==============================] - 0s 2ms/step - loss: 2.8674 - accuracy: 0.9304
Epoch 7/10
124/124 [==============================] - 0s 3ms/step - loss: 2.7140 - accuracy: 0.9321
Epoch 8/10
124/124 [==============================] - 0s 3ms/step - loss: 2.5240 - accuracy: 0.9337
Epoch 9/10
124/124 [==============================] - 0s 3ms/step - loss: 2.6675 - accuracy: 0.9301
Epoch 10/10
124/124 [==============================] - 0s 3ms/step - loss: 2.4275 - accuracy: 0.933

In [168]:
test_size = 1000

testdata=[]
testlabel=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            testdata.append(Token(sol[j:j+2]))
        testlabel+=mark
    except:
        i=i-1

In [169]:
model.evaluate(testdata, testlabel)

125/125 [==============================] - 0s 2ms/step - loss: 2.7060 - accuracy: 0.9225


[2.705986738204956, 0.922457218170166]

In [ ]:
#export model
#
#
#

In [170]:
#Standardizer
LENGTH = 9
def Standardize(a):
    sol = []
    for line in a:
        step = []
        l = len(line)
        
        start = 0
        end = 0
        e = 0
        while end < l:
            while line[end] not in symbol.keys():
                end+=1
                if end >= l:
                    break
            step.append(line[start:end])
            
            if end >= l:
                break
                
            sym = line[end]
            if len(sol) > 0:
                e = len(step)+1
                while sym != sol[0][e]:
                    step.append('')
                    e+=1
                
            step.append(sym)
            
            start=end+1
            end+=1
        
        step += ['']*(LENGTH-len(step))   
        sol.append(step)
    
    return sol

In [176]:
#Practice

formula = "1+2*3-8/4 = 3*3-8/4 = 3*3-2 = 9-2 = 7"
lines = formula.split("=")
for i in range(len(lines)):
    lines[i]=lines[i].strip()

sol = Standardize(lines)
n = len(sol)

print(''.join(sol[0]))

for j in range(1, n-1):
    input_sol = Token(sol[j:j+2])
    result = model.predict([input_sol])
    print('=',''.join(sol[j]), end=' ')
    print("\t\t",np.argmax(result),round(max(result[0])*100,2))

print(''.join(sol[n-1]))

        




1+2*3-8/4
= 3*3-8/4 		 1 100.0
= 3*3-2 		 1 100.0
= 9-2 		 1 100.0
7
